# ETL `homicidios_combinado`

Como lo dice el titulo, en este notebook vamos a combinar los dataframes `hechos_homicidios` y `victimas_homicidios` con el objetivo de tener un dataframe con toda la informacion necesaria para hacer el EDA. 

### Importamos librerias necesarias

In [1]:
import pandas as pd 
import numpy as np

### Procedemos con la carga de los dataframes

In [2]:
#cargamos el dataframe `hechos_homicidios`
hechos_homicidios = pd.read_csv('datasets_limpios/hechos_homicidios.csv')
#cargamos el dataframe `victimas_homicidios`
victimas_homicidios = pd.read_csv('datasets_limpios/victimas_homicidios.csv')

In [3]:
hechos_homicidios.head(1) #visualizamos que el dataframe se haya cargado de forma exitosa. 

,ID,N_VICTIMAS,FECHA,AAAA,MM,FRANJA_HORARIA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,DIRECCIÓN NORMALIZADA,COMUNA,XY (CABA),POS X,POS Y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO


In [4]:
victimas_homicidios.head(1) #visualizamos que el dataframe se haya cargado de forma exitosa. 

,ID,FECHA,AAAA,MM,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00


Una vez cargados ambos datasets, procedemos a combinarlos con el metodo merge. 

In [5]:
homicidios_combinado = hechos_homicidios.merge(victimas_homicidios, on='ID', how='inner') #se fusionan los DataFrames 'hechos_homicidios' y 'victimas_homicidios' en función de los valores de la columna 'ID'.
homicidios_combinado.head(2)                                                              #el parametro how='inner' lo usamos para que solo conserve las filas que tengan un valor coincidente en la columna 'ID' en ambos DataFrames

,ID,N_VICTIMAS,FECHA_x,AAAA_x,MM_x,FRANJA_HORARIA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,DIRECCIÓN NORMALIZADA,...,VICTIMA_x,ACUSADO,FECHA_y,AAAA_y,MM_y,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,2016,1,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",...,MOTO,AUTO,2016-01-01,2016,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,2016,1,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",...,AUTO,PASAJEROS,2016-01-02,2016,1,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


In [6]:
homicidios_combinado.shape #vemos como esta conformado el nuevo df

(717, 25)

Procedemos a crear una funcion para encontrar las columnas que sean iguales en `homicidios_combinado`, para luego eliminarlas.

In [7]:
def encontrar_columnas_iguales(df):
    columnas_diferentes = []
    columnas_iguales = []

    for columna1 in df.columns:
        for columna2 in df.columns:
            if columna1 != columna2:
                if (df[columna1] == df[columna2]).all():
                    columnas_iguales.append((columna1, columna2))
                elif not (df[columna1] == df[columna2]).all():
                    columnas_diferentes.append((columna1, columna2))

    return columnas_iguales, columnas_diferentes


columnas_iguales, columnas_diferentes = encontrar_columnas_iguales(homicidios_combinado[homicidios_combinado.columns])

print("Pares de columnas iguales:")
for columna1, columna2 in columnas_iguales:
    print(f"{columna1} y {columna2} son iguales")

Pares de columnas iguales:
AAAA_x y AAAA_y son iguales
MM_x y MM_y son iguales
AAAA_y y AAAA_x son iguales
MM_y y MM_x son iguales


Como podemos ver, las columnas 'AAAA_x' y 'AAAA_y' son iguales, por lo que procedemos a borrar la segunda ('AAAA_y').
Lo mismo sucede con las columnas 'MM_x' y 'MM_y', borramos la segunda columna ('MM_y').

In [8]:
homicidios_combinado = homicidios_combinado.drop(columns=['AAAA_y', 'MM_y'], axis=1)
homicidios_combinado.head(1)

,ID,N_VICTIMAS,FECHA_x,AAAA_x,MM_x,FRANJA_HORARIA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,DIRECCIÓN NORMALIZADA,...,POS Y,PARTICIPANTES,VICTIMA_x,ACUSADO,FECHA_y,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,2016,1,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",...,-34.68757022,MOTO-AUTO,MOTO,AUTO,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00


Una vez eliminadas las columnas 'AAAA_y' y 'MM_y', procedemos a cambiar el nombre de la columna 'AAAA_x' por 'AAAA' y el de la columna 'MM_x' por 'MM'.

In [9]:
homicidios_combinado = homicidios_combinado.rename(columns={'AAAA_x': 'AAAA'}) #Renombramos la columna 'AAAA_x' por 'AAAA'

In [10]:
homicidios_combinado =  homicidios_combinado.rename(columns={'MM_x': 'MM'}) #Renombramos la columna 'MM_x' por 'MM'

In [11]:
homicidios_combinado.columns

Index(['ID', 'N_VICTIMAS', 'FECHA_x', 'AAAA', 'MM', 'FRANJA_HORARIA',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'CALLE', 'DIRECCIÓN NORMALIZADA',
       'COMUNA', 'XY (CABA)', 'POS X', 'POS Y', 'PARTICIPANTES', 'VICTIMA_x',
       'ACUSADO', 'FECHA_y', 'ROL', 'VICTIMA_y', 'SEXO', 'EDAD',
       'FECHA_FALLECIMIENTO'],
      dtype='object')

Verificamos porque las columnas 'FECHA'  y 'VICTIMA' no son iguales. Para esto, vamos a usar un bucle `for` que itere sobre las filas de dichas columnas y nos devuelva el indice donde estas no son iguales.

In [12]:
indices=[]
for index, row  in homicidios_combinado[['FECHA_x', 'FECHA_y', 'VICTIMA_x', 'VICTIMA_y']].iterrows():

    if row['FECHA_x'] !=row['FECHA_y']:
        indices.append(index)

    if row['VICTIMA_x'] !=row['VICTIMA_y']:
        indices.append(index)

In [13]:
indices # llamamos a la lista 'indices' para ver que indices son diferentes en las columnas anteriormente mencionadas.

[65, 204, 237, 238, 602, 603]

In [14]:
homicidios_combinado[['FECHA_x','FECHA_y', 'VICTIMA_x', 'VICTIMA_y', 'FECHA_FALLECIMIENTO', 'ACUSADO']].iloc[indices]

,FECHA_x,FECHA_y,VICTIMA_x,VICTIMA_y,FECHA_FALLECIMIENTO,ACUSADO
65,2016-07-02,2016-07-03,MOTO,MOTO,2016-07-03 00:00:00,OBJETO FIJO
204,2017-05-24,2017-05-29,MOTO,MOTO,2017-05-29 00:00:00,SD
237,2017-09-02,2017-09-02,OBJETO FIJO,AUTO,2017-09-02 00:00:00,AUTO
238,2017-09-02,2017-09-02,OBJETO FIJO,AUTO,2017-09-02 00:00:00,AUTO
602,2020-12-05,2020-12-05,PEATON_MOTO,PEATON,2020-12-05 00:00:00,MOTO
603,2020-12-05,2020-12-05,PEATON_MOTO,MOTO,2020-12-05 00:00:00,MOTO


Como podemos ver en la tabla de arriba, la columna 'FECHA_y' contiene los valores de la fecha de fallecimiento de la victima, por lo que se decide eliminar dicha columna. En el caso de la columna 'VICTIMA_x' podemos ver que hay datos que son atipicos ya que dice OBJETO FIJO. Si leemos el PDF 'NOTAS_HOMICIDIOS_SINIESTRO_VIAL' se refiere a 'objeto fijo' cuando el ACUSADO es un objeto inmovil o cuando se produce una perdida de equilibrio de un vehiculo de dos ruedas, por lo tanto este dato lo consideramos atipico ya que si revisamos la columna 'VICTIMA_y' y la columna 'ACUSADO', podemos ver que el vehiculo en cuestion es un auto. Con esto deducimos que el dato de la columna 'VICTIMA_x' corresponde a la columna 'ACUSADO' y se procede a reemplazarlo.

In [15]:
# Encuentra las filas donde 'VICTIMA_x' es igual a 'OBJETO FIJO'
filas_objeto_fijo = homicidios_combinado['VICTIMA_x'] == 'OBJETO FIJO'

# Almacena temporalmente los valores de 'VICTIMA_x' en una columna temporal
homicidios_combinado.loc[filas_objeto_fijo, 'TEMP'] = homicidios_combinado.loc[filas_objeto_fijo, 'VICTIMA_x']

# Intercambia los valores entre 'VICTIMA_x' y 'ACUSADO'
homicidios_combinado.loc[filas_objeto_fijo, 'VICTIMA_x'] = homicidios_combinado.loc[filas_objeto_fijo, 'ACUSADO']
homicidios_combinado.loc[filas_objeto_fijo, 'ACUSADO'] = homicidios_combinado.loc[filas_objeto_fijo, 'TEMP']

# Elimina la columna temporal 'TEMP'
homicidios_combinado.drop(columns=['TEMP'], inplace=True)

In [16]:
homicidios_combinado[['FECHA_x', 'VICTIMA_x',  'VICTIMA_y','FECHA_FALLECIMIENTO','ACUSADO']].iloc[indices]

,FECHA_x,VICTIMA_x,VICTIMA_y,FECHA_FALLECIMIENTO,ACUSADO
65,2016-07-02,MOTO,MOTO,2016-07-03 00:00:00,OBJETO FIJO
204,2017-05-24,MOTO,MOTO,2017-05-29 00:00:00,SD
237,2017-09-02,AUTO,AUTO,2017-09-02 00:00:00,OBJETO FIJO
238,2017-09-02,AUTO,AUTO,2017-09-02 00:00:00,OBJETO FIJO
602,2020-12-05,PEATON_MOTO,PEATON,2020-12-05 00:00:00,MOTO
603,2020-12-05,PEATON_MOTO,MOTO,2020-12-05 00:00:00,MOTO


In [17]:
homicidios_combinado.drop(columns='FECHA_y', inplace=True) #Se procede a eliminar la columna 'FECHA_y'

Todavia hay datos diferentes dentro de las columnas 'VICTIMA_x' y 'VICTIMA_y'. Como podemo observar en el indice 602 y 603 hace referencia a 'PEATON-MOTO' lo cual se deduce que hubo dos victimas, pero en la columna 'VICTIMA_y' solo hay una victima y si lo comparamos con la columna 'ACUSADO' podemos deducir que la victima fue un peaton y el acusado una moto. Por esta razon se decide eliminar la columna 'VICTIMA_x'.

In [18]:
homicidios_combinado = homicidios_combinado.drop(columns='VICTIMA_x', axis=1)

In [19]:
homicidios_combinado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     717 non-null    object
 1   N_VICTIMAS             717 non-null    int64 
 2   FECHA_x                717 non-null    object
 3   AAAA                   717 non-null    int64 
 4   MM                     717 non-null    int64 
 5   FRANJA_HORARIA         717 non-null    object
 6   LUGAR_DEL_HECHO        717 non-null    object
 7   TIPO_DE_CALLE          717 non-null    object
 8   CALLE                  716 non-null    object
 9   DIRECCIÓN NORMALIZADA  708 non-null    object
 10  COMUNA                 717 non-null    int64 
 11  XY (CABA)              717 non-null    object
 12  POS X                  717 non-null    object
 13  POS Y                  717 non-null    object
 14  PARTICIPANTES          717 non-null    object
 15  ACUSADO                

Procedemos a cambiar el nombre de las columnas que se vieron afectadas con el merge.

In [20]:
homicidios_combinado.columns

Index(['ID', 'N_VICTIMAS', 'FECHA_x', 'AAAA', 'MM', 'FRANJA_HORARIA',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'CALLE', 'DIRECCIÓN NORMALIZADA',
       'COMUNA', 'XY (CABA)', 'POS X', 'POS Y', 'PARTICIPANTES', 'ACUSADO',
       'ROL', 'VICTIMA_y', 'SEXO', 'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')

In [21]:
homicidios_combinado = homicidios_combinado.rename(columns={'FECHA_x': 'FECHA', 'VICTIMA_y': 'VICTIMA'})

In [22]:
homicidios_combinado.head(2)

,ID,N_VICTIMAS,FECHA,AAAA,MM,FRANJA_HORARIA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,DIRECCIÓN NORMALIZADA,...,XY (CABA),POS X,POS Y,PARTICIPANTES,ACUSADO,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,2016,1,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",...,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,2016,1,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",...,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,PASAJEROS,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


### Enriquecimiento de columnas para los KPI's

**Objetivo:**
* Crear una nueva columna 'SEMESTRE' a partir de la columna 'MM' y luego combinar la columna 'SEMESTRE' con la columna 'AAAA' con el fin de tener una columna del semetre por año.

In [23]:
# Función para clasificar como primer o segundo semestre
def clasificar_semestre(mes):
    if mes <= 6:
        return "1"
    else:
        return "2"

# Aplicar la función a la columna "MM" y crear una nueva columna "Semestre"
homicidios_combinado['SEMESTRE'] = homicidios_combinado['MM'].apply(clasificar_semestre)

# Concatenar con la columna AAAA
homicidios_combinado['AAAA_SEMESTRE'] = homicidios_combinado['AAAA'].astype(str) + "_" + homicidios_combinado['SEMESTRE']

# eliminar columnas irrelevantes
homicidios_combinado = homicidios_combinado.drop(columns=["MM", "SEMESTRE"])

# renombrar columna "AAAA_SEMESTRE"
homicidios_combinado = homicidios_combinado.rename(columns={'AAAA_SEMESTRE': 'SEMESTRE'})

homicidios_combinado.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,FRANJA_HORARIA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,DIRECCIÓN NORMALIZADA,COMUNA,...,POS X,POS Y,PARTICIPANTES,ACUSADO,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,SEMESTRE
0,2016-0001,1,2016-01-01,2016,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,...,-58.47533969,-34.68757022,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00,2016_1
1,2016-0002,1,2016-01-02,2016,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,...,-58.50877521,-34.66977709,AUTO-PASAJEROS,PASAJEROS,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00,2016_1
2,2016-0003,1,2016-01-03,2016,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,1,...,-58.39040293,-34.63189362,MOTO-AUTO,AUTO,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00,2016_1


### Procedemos a guardar `homicidios_combinado` a formato CSV

In [24]:
homicidios_combinado.to_csv('datasets_limpios/homicidios_combinado.csv', index=False, encoding='utf-8')